# RGAT

## Prep

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!pip install sentence-transformers
!pip install rdflib
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

! pip install torchmetrics

In [ ]:
import torch.nn.functional as F
import os
import torch
import numpy as np
import random


from typing import List, Callable

from torch import Tensor, device
import pandas as pd

In [ ]:
#os.chdir('/pfs/data5/home/ma/ma_ma/ma_eweinsto/3_MAGNET')
os.chdir("/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/all_in/MAGNET")

In [ ]:
import models.helper_funcs as helper_funcs
import models.models as models
import models.losses as losses_funcs

## Data

In [ ]:
os.chdir("/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/!old/prepared_data")

In [ ]:
#preprocessed_data_path = "../2_data/2_preprocessed"
#use_case = "anatomy"
#a_or_b = "case_b"
#data = torch.load(preprocessed_data_path + "/" + use_case +  "/" + a_or_b + ".pt")
#double_left_tensor = torch.load(preprocessed_data_path+"/"+use_case+"/a/double_left_tensor.pt")
#double_right_tensor = torch.load(preprocessed_data_path+"/"+use_case+"/a/double_right_tensor.pt")

In [ ]:
data = torch.load("og_anatomy.pt")

In [ ]:
data

Data(x_one=[3105, 384], edge_index_one=[2, 5957], edge_type_one=[5957], edge_attr_one=[5957, 384], x_two=[9423, 384], edge_index_two=[2, 13196], edge_type_two=[13196], edge_attr_two=[13196, 384], left_indices=[1516], right_indices=[1516], train_set_left=[1061], test_set_left=[303], val_set_left=[152], train_set_right=[1061], test_set_right=[303], val_set_right=[152])

Model

In [ ]:
#os.chdir('/pfs/data5/home/ma/ma_ma/ma_eweinsto/1_notebooks/models')
os.chdir("/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/all_in/MAGNET")

Other adjustments

In [ ]:
num_edge_types = max(data.edge_type_one.max(), data.edge_type_two.max()).item() + 1

In [ ]:
double_left_tensor, double_right_tensor = helper_funcs.create_fake_data(data.train_set_right, data.train_set_left)

In [ ]:
device = 'cpu' 
source_data = data.to(device)
model = models.RGATClassic(384, 384, num_edge_types).to(device)
myloss = losses_funcs.CosineSimilarityLoss1(model)


optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.0005, momentum=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)

def train():
    model.train()
    optimizer.zero_grad()
    #loss = myloss(source_data, matches) # version 0, 2
    loss = myloss(source_data, matches, double_left_tensor, double_right_tensor)
    loss.backward()
    optimizer.step()
    return float(loss)

In [ ]:
precisions = []
recalls = []
losses = []

precision_tests = []
recall_tests = []

wrong_sets_left_train = []
wrong_sets_right_train = []
matches = []

for epoch in range(1, 1000):
    loss = train()
    prec, rec, matches = helper_funcs.test_harder('train', model, source_data)
    prec_test, rec_test, matches_test = helper_funcs.test_harder('test', model, source_data)

    precisions.append(prec)
    recalls.append(rec)
    losses.append(loss)
    precision_tests.append(prec_test)
    recall_tests.append(rec_test)
    
    print(f'TRAINING Epoch: {epoch:02d}, Loss: {loss:.4f}, Precision: {prec:.4f} , Recall: {rec:.4f} ')
    print(f'TEST Epoch: {epoch:02d}, Precision: {prec_test:.4f} , Recall: {rec_test:.4f} ')
    x, y, wrong_sets_left, wrong_sets_right = helper_funcs.filter_out_wrong_matches(matches, data)
    wrong_sets_left_train.append(wrong_sets_left)
    wrong_sets_right_train.append(wrong_sets_right)

Save output

In [ ]:
version = 'V1'
helper_funcs.save_wrong_matches_left(version, wrong_sets_left_train)
helper_funcs.save_wrong_matches_right(version, wrong_sets_right_train)

In [ ]:
model_name = 'RGATConv'
save_training_verlauf(losses, precisions, recalls,
                      precision_tests, recall_tests,
                      use_case, a_or_b, model_name, version)

In [ ]:
#save_model(model_name, version, model)